### Sensitive analysis:


In [2]:
from sast.sast import *
from sktime.datasets import load_UCR_UEA_dataset
from sktime.classification.kernel_based import RocketClassifier
import time
import numpy as np
import pandas as pd
from sast.sast import znormalize_array,from_2d_array_to_nested
import matplotlib.pyplot as plt
from sktime.datasets import tsc_dataset_names

In [3]:
ds_sens = tsc_dataset_names.univariate_equal_length
ds_sens = ["Chinatown","BirdChicken","Coffee"]
max_ds=len(ds_sens) #exploring dataset in UEA & UCR Time Series Classification Repository
#print(ds_sens)
print(ds_sens)


['Chinatown', 'BirdChicken', 'Coffee']


In [4]:
#ds='Chinatown' # Chosing a dataset from # Number of classes to consider
runs = 1
range_total = [1 ,10 , 100]
#list
df_result = {}
list_score = []
list_time = []
list_dataset = []
list_hyperparameter = []
list_method = []
list_rpoint = []
list_nb_per_class = []

for ds in ds_sens:
    X_train, y_train = load_UCR_UEA_dataset(name=ds, extract_path='data', split="train", return_type="numpy2d")
    X_test, y_test = load_UCR_UEA_dataset(name=ds, extract_path='data', split="test", return_type="numpy2d")
    for i in range(runs):
        start = time.time()
        rocket= RocketClassifier(num_kernels=10_000)
        rocket.fit(X_train,y_train)
        end = time.time()
        list_score.append(rocket.score(X_test,y_test))
        list_time.append(end-start)
        list_dataset.append(ds)
        list_hyperparameter.append("num_kernels=10_000")
        list_rpoint.append(str(0))
        list_nb_per_class.append(str(0))
        list_method.append("Rocket")
        for p in range_total:
            for g in range_total:
                if p ==0:
                    p=1
                if g ==0:
                    g=1
                start = time.time()
                random_state = None
                rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=g, classifier=RidgeClassifierCV())
                rsast_ridge.fit(X_train, y_train)
                end = time.time()
                list_score.append(rsast_ridge.score(X_test,y_test))
                list_time.append(end-start)
                list_dataset.append(ds)
                list_hyperparameter.append("n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                list_rpoint.append(str(p))
                list_nb_per_class.append(str(g))
                list_method.append("Rsast")
                print("n_random_points="+str(p)+" nb_inst_per_class="+str(g))

    df_result['score']=list_score
    df_result['time']=list_time
    df_result['dataset']=list_dataset
    df_result['hyperparameter']=list_hyperparameter
    df_result['rpoint']=list_rpoint
    df_result['nb_per_class']=list_nb_per_class
    df_result['method']=list_method

n_random_points=1 nb_inst_per_class=1
n_random_points=1 nb_inst_per_class=10
n_random_points=1 nb_inst_per_class=100
n_random_points=10 nb_inst_per_class=1
n_random_points=10 nb_inst_per_class=10
n_random_points=10 nb_inst_per_class=100
n_random_points=100 nb_inst_per_class=1
n_random_points=100 nb_inst_per_class=10
n_random_points=100 nb_inst_per_class=100
n_random_points=1 nb_inst_per_class=1


In [ ]:
df_result=pd.DataFrame(df_result)
df_result.to_excel("df_tunning_hyper.xlsx") 
df_result

,score,time,dataset,hyperparameter,rpoint,nb_per_class,method
0,0.982507,7.969090,Chinatown,num_kernels=10_000,0,0,Rocket
1,0.877551,0.012979,Chinatown,n_random_points=1 nb_inst_per_class=1,1,1,Rsast
2,0.970845,0.047904,Chinatown,n_random_points=1 nb_inst_per_class=10,1,10,Rsast
3,0.970845,0.016964,Chinatown,n_random_points=10 nb_inst_per_class=1,10,1,Rsast
4,0.976676,0.114693,Chinatown,n_random_points=10 nb_inst_per_class=10,10,10,Rsast
5,0.982507,7.296936,Chinatown,num_kernels=10_000,0,0,Rocket
6,0.918367,0.012927,Chinatown,n_random_points=1 nb_inst_per_class=1,1,1,Rsast
7,0.921283,0.045878,Chinatown,n_random_points=1 nb_inst_per_class=10,1,10,Rsast
8,0.778426,0.017971,Chinatown,n_random_points=10 nb_inst_per_class=1,10,1,Rsast
9,0.976676,0.112698,Chinatown,n_random_points=10 nb_inst_per_class=10,10,10,Rsast
